# Map of Kotlin Island with Inset

Kotlin Island is situated in the Gulf of Finland and is one of the districts of the city of Saint Petersburg (Russia).


In [1]:
%useLatestDescriptors
%use lets-plot
%use lets-plot-gt

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.11.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.7.0.

#### All Districts of St.Petersburg

In [3]:
import java.net.URL
import org.geotools.geojson.feature.FeatureJSON

fun getSpatialDataset(url: String): SpatialDataset {
    return FeatureJSON()
        .readFeatureCollection(URL(url).readText()) // Read GeoJSON file.
        .toSpatialDataset() // Transform GeoTools `FeatureCollection` to Lets-Plot `SpatialDataset`.
}

In [4]:
// The districts boundaries.
val districtsSD = getSpatialDataset("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/spb_districts.geojson")


In [5]:
// Kotlin Island bounding box.
val xmin = 29.63
val xmax = 29.815
val ymin = 59.965
val ymax = 60.035


In [6]:
val pDistricts = (letsPlot()
      + geomPolygon(map = districtsSD, color = "#a1d99b", fill = "#f7fcf5")
      + geomRect(xmin = xmin, ymin = ymin, xmax = xmax, ymax = ymax, color = "red", alpha = 0)
      + geomText(label = "Saint Petersburg", x = 30.334445, y = 59.934294, color = "black", size = 6)
      + themeVoid() + theme(plotInset = 0, plotBackground = elementRect(size = 1))
)    
pDistricts + coordMap()

#### Detailed Map of Kotlin Island


In [7]:
// Tourist attractions with coordinates.
val placesSD = getSpatialDataset("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/kotlin_places.geojson")
placesSD

SpatialDataset(GEOJSON, key='geometry', map={name=[Kronstadt Naval Cathedral, Kronstadt History Museum, Kronstadt Naval Museum, City Russian Cemetery, Kronstadt Lutheran Cemetery, Vladimir Church], type=[church, museum, museum, cemetery, cemetery, church], geometry=[{"type":"Point","coordinates":[29.777455,59.991744]}, {"type":"Point","coordinates":[29.791317,59.986777]}, {"type":"Point","coordinates":[29.763422,59.996108]}, {"type":"Point","coordinates":[29.70613,60.019788]}, {"type":"Point","coordinates":[29.749861,60.002212]}, {"type":"Point","coordinates":[29.766254,59.998515]}]}, crs=null)

In [8]:
val pKotlin = (letsPlot()
      // Zoom-in the map.               
      + coordCartesian(xlim = (xmin to xmax), ylim = (ymin to ymax))
      + geomPolygon(map = districtsSD, color = "#31a354", fill = "#e5f5e0")

      // Add the points of interest.
      + geomPoint(map = placesSD, size = 5) { color = "type"; shape = "type" }
      + geomText(map = placesSD, hjust = "right", nudgeX = -5, nudgeUnit = "size") { label = "name" }

      // Add annotation texts.
      + geomText(label = "Kotlin Isl.", x = 29.725, y = 60.011, color = "#31a354", size = 13, fontface = "italic")
      + geomText(label = "Gulf of Finland", x = 29.665, y = 60.002, color = "#578bcc", size = 11, fontface = "italic")
      
      + themeVoid() + theme(
          plotBackground = elementRect(fill = "#F0F7FE", size = 1),
          plotInset = 0,
          legendTitle = "blank",
          legendBackground = "blank"
      ).legendPosition(.1, .12)
)

pKotlin

#### Combining Both Plots in One Figure

In [9]:
(ggbunch(
    plots = listOf(pKotlin, pDistricts),
    regions = listOf(
            listOf(0, 0, 1, 1),
            listOf(0.75, 0, 0.25, 0.37, -10, 10)
        )
    )
  + ggtitle("Tourist Attractions on Kotlin Island")
  + theme(plotTitle = elementText(hjust = 0.5, face = "bold", size = 16))
)